In [1]:
import numpy as np
import pandas as pd
import os
import glob

# webscraping
import json
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlopen
import os

In [2]:
path = os.getcwd()
home_path = os.path.dirname(path)
data_folder = f'{home_path}\\Data'

In [3]:
files = glob.glob(f"{data_folder}\\*.txt")
files

['C:\\Users\\david\\Documents\\@@ Python\\Jaar 5\\Q3\\ENVM1502\\Git\\ENVM1502-Catchment-model\\Data\\120_days_01310521.txt',
 'C:\\Users\\david\\Documents\\@@ Python\\Jaar 5\\Q3\\ENVM1502\\Git\\ENVM1502-Catchment-model\\Data\\2007-2010_01310521.txt',
 'C:\\Users\\david\\Documents\\@@ Python\\Jaar 5\\Q3\\ENVM1502\\Git\\ENVM1502-Catchment-model\\Data\\station_ids.txt']

### station information:
station generated by considering all station in the same hydraulic unit

In [9]:
ids = np.loadtxt(f'{data_folder}\\station_ids.txt',dtype=int)

In [51]:
ids

array([131199050,   1312000,   1315000,   1315170,   1315226,   1315227,
         1315500,   1318500,   1327500,   1327750,   1328770,   1329490,
         1329500,   1330000,   1330500,   1334500,   1335754,   1335755,
         1358000,   1359133,   1359135,   1359528,   1359750,   1360640,
         1361000,   1361200,   1362090,   1362192, 136219503,  13621955,
         1362200,   1362230,   1362297, 136230002,   1362322,   1362336,
         1362342,   1362357,   1362368,   1362370,   1362380,   1362465,
         1362487,   1362497,   1362500,   1363382,   1363556,   1364500,
         1372051,   1372500, 137449480,   1374505,   1374531,   1374559,
         1374581,   1374598, 137462010,   1374654,   1374701,   1374781,
         1374821,   1374890,   1374901,   1374918,   1374930,   1374941,
         1374976,   1374987, 137499350,   1375000])

In [70]:
begin_end_date_csv = "Ids_with_begin-end-date.csv"
if os.path.exists(begin_end_date_csv):
    df_id_dates = pd.read_csv(begin_end_date_csv,index_col=0)
else:
    lst_begin_date = []
    lst_end_date = []
    lst_no_flow    = []
    for i in ids:
        url_start_table = "https://waterdata.usgs.gov/nwis/inventory/?site_no="
        end = "&agency_cd=USGS"
        site = url_start_table + f'0{i}' + end
        page = urlopen(site)
        html_bytes = page.read()
        html = html_bytes.decode("utf-8")
        table = html[html.find('<div id="stationTable">'):html.find("</table>")]
        table_data = [[cell.text.strip() for cell in row("td")]
                                 for row in BeautifulSoup(table)("tr")]
        headers = [[cell.text.strip() for cell in row("th")]
                              for row in BeautifulSoup(table)("tr")][0]

        df_station = pd.DataFrame(data=table_data, columns=headers)
        try:
            series_begin_date = df_station.set_index("Data Type").loc["Discharge, cubic feet per second","Begin Date"]
            series_end_date = df_station.set_index("Data Type").loc["Discharge, cubic feet per second","End Date"]

            if type(series_begin_date) == str:
                begin_date = series_begin_date
                end_date   = series_end_date
            else:
                begin_date = series_begin_date.iloc[0]
                end_date = series_end_date.iloc[0]

            lst_begin_date.append(begin_date)
            lst_end_date.append(end_date)
        except KeyError:
            lst_no_flow.append(i)
    
    df_id_dates = pd.DataFrame(data=list(zip(ids,lst_begin_date,lst_end_date)), columns=["ID","Begin Date","End Date"])
    df_id_dates.to_csv(begin_end_date_csv)

In [71]:
df_id_dates

,ID,Begin Date,End Date
0,131199050,2007-01-25,2010-05-09
1,1312000,1925-10-01,2023-03-21
2,1315000,1912-07-01,2023-03-21
3,1315170,2016-09-30,2022-12-30
4,1315226,2016-09-30,2023-02-20
...,...,...,...
64,1374930,1994-03-23,2023-03-21
65,1374941,1995-12-08,2005-09-29
66,1374976,1995-10-21,2009-06-29
67,1374987,1999-06-11,2009-06-29


#### Hourly: 

too specific

In [48]:
url_start = "https://waterservices.usgs.gov/nwis/iv/?sites="
pram = "&parameterCd=00060"
start = "&startDT="
t0 = "2023-03-15T05:45:18.716-04:00"
end_time = "&endDT="
tend = "2023-03-22T05:45:18.716-04:00"
post_fix = "&siteStatus=all&format=rdb"

# Daily

In [73]:
url_start_daily = "https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no="
begin_date = "&legacy=&referred_module=sw&period=&begin_date="
end_data = "&end_date="

In [157]:
df_id_dates.loc[17]

ID               1335755
Begin Date    1946-02-21
End Date      2023-03-21
Name: 17, dtype: object

In [183]:
lst_data = []
lst_errors = []
for i in df_id_dates.index:
#     if i == 19:
    site = url_start_daily + f'0{df_id_dates.loc[i,"ID"]}' + begin_date + df_id_dates.loc[i,"Begin Date"] \
                                                           + end_data + df_id_dates.loc[i,"End Date"]
    print(i,end="\r")
    page = urlopen(site)
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")
    if html[:19] == "No sites/data found":
        lst_errors.append([i,df_id_dates.loc[i,"ID"]])
        pass
    elif len(html) < 1500:
        lst_errors.append([i,df_id_dates.loc[i,"ID"]])
        pass
    else:
        data = html[html.find("\n# \n")+4:].split('\n')
        header = data[0].split("\t")
        if header[0] == "agency_cd":
            header = ['agency_cd', 'site_no', 'datetime', 'Q_ft3_s_mean',
               'Data codes']
        else: 
            print(i, header)
        rows = data[2:]
        df = pd.DataFrame(data=[row.split('\t') for row in rows],columns=header)
        df.replace({"":pd.NA},inplace=True)
        df.dropna(inplace=True)
        df["Q_m3_s_mean"] = df.apply(lambda x: float(x['Q_ft3_s_mean']) * 0.0283168466\
                                     if x['Q_ft3_s_mean'][0].isnumeric()
                                     else x, axis=1)
        df_out = df[['agency_cd', 'site_no', 'datetime', 'Data codes', 'Q_m3_s_mean']]
        lst_data.append(df_out)

26 ['--------------------------------- WARNING ----------------------------------------']


ValueError: 1 columns passed, passed data had 4 columns

In [152]:
for df in lst_data:
    df.to_csv(f'{data_folder}\\{df.iloc[0]["site_no"]}.csv')

[     agency_cd     site_no    datetime Data codes  Q_m3_s_mean
 0         USGS  0131199050  2007-01-25          A     1.019406
 1         USGS  0131199050  2007-01-26          A     0.903307
 2         USGS  0131199050  2007-01-27          A     0.858000
 3         USGS  0131199050  2007-01-28          A     0.858000
 4         USGS  0131199050  2007-01-29          A     0.835347
 ...        ...         ...         ...        ...          ...
 1196      USGS  0131199050  2010-05-05          A     1.220456
 1197      USGS  0131199050  2010-05-06          A     1.277090
 1198      USGS  0131199050  2010-05-07          A     1.231783
 1199      USGS  0131199050  2010-05-08          A     1.667862
 1200      USGS  0131199050  2010-05-09          A     2.718417
 
 [1201 rows x 5 columns]]